In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import nltk
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# The goal of this prototype is to fetch the lyrics of a specific song, in preparation for the full script that will iterate through the yearly data.

songdata = pd.read_csv('data/2019top10songs.csv')

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
cookies = { 'sessionid': '123..'}
i = 5 # Set i to the position in the songdata datafrae you would like to examine.
url = 'https://www.azlyrics.com/lyrics/'
artist = songdata['Artist'][i] # artist1 and artist2 are the most common ways azlyrics represents artist names. artist1 removes spaces while artist2 replaces                                  them with a dash. artist3 and artist4 handle the way that Wikipedia presents collaborations.
artist1 = ''.join(artist.split()).lower().replace('\'','').replace(',','').replace('!','')
artist2 = artist.lower().replace('\'','').replace(',','').replace(' ','-').replace('!','')
artist3 = artist1[:artist1.rfind('and')]
artist4 = artist2[:artist2.rfind('-and')]
title = ''.join(songdata['Title'][i].split()).lower() # azlyrics seems to represent all song titles in the same way.
if requests.get(url + artist1 + '/' + title + '.html').status_code == 200:
    html = requests.get(url + artist1 + '/' + title + '.html',cookies=cookies,headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    print(soup.prettify()[0:1000])
elif requests.get(url + artist2 + '/' + title + '.html').status_code == 200:
    html = requests.get(url + artist2 + '/' + title + '.html').text
    soup = BeautifulSoup(html, 'lxml')
    print(soup.prettify()[0:1000])
elif requests.get(url + artist3 + '/' + title + '.html').status_code == 200:
    html = requests.get(url + artist3 + '/' + title + '.html').text
    soup = BeautifulSoup(html, 'lxml')
    print(soup.prettify()[0:1000])
elif requests.get(url + artist4 + '/' + title + '.html').status_code == 200:
    html = requests.get(url + artist4 + '/' + title + '.html').text
    soup = BeautifulSoup(html, 'lxml')
    print(soup.prettify()[0:1000])
else:
    print('Song ' + songdata['Title'][i] + ' by ' + artist + ' at position ' + str(i) + ' not found!')

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Panic! At The Disco &quot;High Hopes&quot;: (High, high hopes) Had to have high, high hopes for a living Shooting for the stars when I couldn't..." name="description"/>
  <meta content="High Hopes lyrics, Panic! At The Disco High Hopes lyrics, Panic! At The Disco lyrics" name="keywords"/>
  <meta content="noarchive" name="robots"/>
  <meta content="//www.azlyrics.com/az_logo_tr.png" property="og:image"/>
  <title>
   Panic! At The Disco - High Hopes Lyrics | AZLyrics.com
  </title>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="//www.azlyrics.com/bsaz.css?4" rel="stylesheet"/>
  <!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
  <!--[if lt IE 9]>
<script src="https://o


In [4]:
artist1

'panicatthedisco'

In [5]:
my_text = soup.find('div',attrs={'class': None}).text.replace('\n',' ')[3:] # Extract lyrics text from the azlyrics page and replace newlines with spaces. Also, eliminate the \r tag and space at the beginning.
try:
    my_text = my_text[:my_text.rindex("[")-1] # Remove miscellaneous notes at the end of lyrics.
except ValueError:
    my_text = my_text
my_text = word_tokenize(my_text)
my_text = [i.lower() for i in my_text if not re.fullmatch('[' + string.punctuation + ']+', i)] #Remove tokens that are just punctuation.

In [6]:
port_stem = PorterStemmer() # The following code will stem the words in the lyrics. This means that, for example, "stop", "stopping", and "stops" will only count as one unique word.

stem_text = []

for i in range(len(my_text)):
    stem_text.append(port_stem.stem(my_text[i]))

print(stem_text)

['high', 'high', 'hope', 'had', 'to', 'have', 'high', 'high', 'hope', 'for', 'a', 'live', 'shoot', 'for', 'the', 'star', 'when', 'i', 'could', "n't", 'make', 'a', 'kill', 'did', "n't", 'have', 'a', 'dime', 'but', 'i', 'alway', 'had', 'a', 'vision', 'alway', 'had', 'high', 'high', 'hope', 'high', 'high', 'hope', 'had', 'to', 'have', 'high', 'high', 'hope', 'for', 'a', 'live', 'did', "n't", 'know', 'how', 'but', 'i', 'alway', 'had', 'a', 'feel', 'i', 'wa', 'gon', 'na', 'be', 'that', 'one', 'in', 'a', 'million', 'alway', 'had', 'high', 'high', 'hope', 'mama', 'said', 'fulfil', 'the', 'propheci', 'be', 'someth', 'greater', 'go', 'make', 'a', 'legaci', 'manifest', 'destini', 'back', 'in', 'the', 'day', 'we', 'want', 'everyth', 'want', 'everyth', 'mama', 'said', 'burn', 'your', 'biographi', 'rewrit', 'your', 'histori', 'light', 'up', 'your', 'wildest', 'dream', 'museum', 'victori', 'everi', 'day', 'we', 'want', 'everyth', 'want', 'everyth', 'mama', 'said', 'do', "n't", 'give', 'up', 'it', "'

In [8]:
unique_count = len(set(stem_text))
unique_count

107

In [11]:
len(my_text)

484